In [3]:
# Import libraries
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)
import requests
import matplotlib.pyplot as plt
import json
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
# # Read csv and create dataframe
# from google.colab import files
# uploaded = files.upload()

In [10]:
import io
salaries_df = pd.read_csv('Resources/salaries_cleaned.csv')

In [28]:
# from google.colab import files
# uploaded = files.upload()

In [11]:
salaries_df.head()

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education
0,0,2017-06-07 11:33:00,2017-06-07 00:00:00,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown
1,1,2017-06-10 17:11:00,2017-06-10 00:00:00,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown
2,2,2017-06-11 14:53:00,2017-06-11 00:00:00,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown
3,3,2017-06-17 00:23:00,2017-06-17 00:00:00,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown
4,4,2017-06-20 10:58:00,2017-06-20 00:00:00,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown


In [12]:
salaries_df.columns

Index(['Unnamed: 0', 'timestamp', 'date', 'company', 'level', 'title',
       'totalyearlycompensation', 'location', 'latitude', 'longitude',
       'yearsofexperience', 'yearsatcompany', 'basesalary', 'stockgrantvalue',
       'bonus', 'gender', 'Race', 'Education'],
      dtype='object')

In [13]:
salaries_df['company'].value_counts()

Amazon                      8126
Microsoft                   5216
Google                      4330
Facebook                    2990
Apple                       2028
                            ... 
Samsung research America       1
Bny Mellon                     1
yelp                           1
Bloomberg lp                   1
tableau software               1
Name: company, Length: 1631, dtype: int64

In [14]:
val=salaries_df['company'].value_counts()
print(val[val>200])

Amazon               8126
Microsoft            5216
Google               4330
Facebook             2990
Apple                2028
Oracle               1128
Salesforce           1056
Intel                 949
Cisco                 907
IBM                   907
Uber                  880
Capital One           778
LinkedIn              701
VMware                657
Qualcomm              565
JPMorgan Chase        541
Bloomberg             537
Goldman Sachs         469
Intuit                449
PayPal                426
Deloitte              401
Twitter               395
Adobe                 383
Walmart Labs          381
Nvidia                357
Accenture             355
Expedia               352
Wayfair               338
eBay                  338
SAP                   330
Lyft                  325
Netflix               268
Workday               262
Dropbox               254
Shopify               234
Airbnb                229
Dell Technologies     228
Atlassian             219
Visa        

In [15]:
#top 20 companies
comapanies_to_replace=[]
company=salaries_df['company'].value_counts()
company=pd.DataFrame(company)
company.head(20)

,company
Amazon,8126
Microsoft,5216
Google,4330
Facebook,2990
Apple,2028
Oracle,1128
Salesforce,1056
Intel,949
Cisco,907
IBM,907


In [16]:
comapanies_to_replace.append(company[company.company<430].index)

AttributeError: 'list' object has no attribute 'value_counts'

In [17]:
#Replaced with other for all companies besides the top 20
for x in comapanies_to_replace:
    salaries_df['company'] = salaries_df['company'].replace(x,"Other")
    
# Check to make sure binning was successful
salaries_df['company'].value_counts()

Other             29423
Amazon             8126
Microsoft          5216
Google             4330
Facebook           2990
Apple              2028
Oracle             1128
Salesforce         1056
Intel               949
Cisco               907
IBM                 907
Uber                880
Capital One         778
LinkedIn            701
VMware              657
Qualcomm            565
JPMorgan Chase      541
Bloomberg           537
Goldman Sachs       469
Intuit              449
Name: company, dtype: int64

In [53]:
reduced_df = salaries_df[[ 'company', 'title', 'totalyearlycompensation',
       'yearsofexperience', 'yearsatcompany', 'latitude', 'longitude']]

In [54]:
reduced_df


,company,title,totalyearlycompensation,yearsofexperience,yearsatcompany,latitude,longitude
0,Oracle,Product Manager,127000,1.5,1.5,37.486324,-122.232523
1,Other,Software Engineer,100000,5.0,3.0,37.779026,-122.419906
2,Amazon,Product Manager,310000,8.0,0.0,47.603832,-122.330062
3,Apple,Software Engineering Manager,372000,7.0,5.0,37.368830,-122.036349
4,Microsoft,Software Engineer,157000,5.0,3.0,37.389389,-122.083210
...,...,...,...,...,...,...,...
62637,Google,Software Engineer,327000,10.0,1.0,47.603832,-122.330062
62638,Microsoft,Software Engineer,237000,2.0,2.0,47.669414,-122.123877
62639,Other,Software Engineer,220000,14.0,12.0,47.603832,-122.330062
62640,Salesforce,Software Engineer,280000,8.0,4.0,37.779026,-122.419906


In [55]:
dummies_df = pd.get_dummies(reduced_df)
dummies_df.head()

,totalyearlycompensation,yearsofexperience,yearsatcompany,latitude,longitude,company_Amazon,company_Apple,company_Bloomberg,company_Capital One,company_Cisco,...,title_Marketing,title_Mechanical Engineer,title_Product Designer,title_Product Manager,title_Recruiter,title_Sales,title_Software Engineer,title_Software Engineering Manager,title_Solution Architect,title_Technical Program Manager
0,127000,1.5,1.5,37.486324,-122.232523,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,100000,5.0,3.0,37.779026,-122.419906,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,310000,8.0,0.0,47.603832,-122.330062,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,372000,7.0,5.0,37.368830,-122.036349,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,157000,5.0,3.0,37.389389,-122.083210,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [56]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop(columns=["totalyearlycompensation"]).values
y = dummies_df["totalyearlycompensation"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [33]:
# # Split our preprocessed data into our features and target arrays
# X = reduced_df.drop(columns=["totalyearlycompensation"]).values
# y = reduced_df["totalyearlycompensation"].values

# # Split the preprocessed data into a training and testing dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [57]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [58]:
X.shape

(62642, 39)

In [59]:
#linear regression
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.linear_model import LinearRegression

In [15]:
#model = LinearRegression()

In [ ]:
#y = dummies_df["totalyearlycompensation"].values


In [60]:
#random forest
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

In [61]:
rf_model = RandomForestRegressor(n_estimators=1000, random_state=78)

In [62]:
rf_model = rf_model.fit(X_train_scaled, y_train)

In [63]:
predictions = rf_model.predict(X_test_scaled)

In [36]:
predictions


array([340845.62466422, 198543.30842788, 100942.55048343, ...,
       343942.89365079, 158907.15948986, 201270.2384968 ])

In [39]:
df=pd.DataFrame( predictions, y_test)
df

,0
220000,340845.624664
146000,198543.308428
114000,100942.550483
200000,385002.619447
206000,273411.963343
...,...
113000,198543.308428
168000,178423.488918
275000,343942.893651
220000,158907.159490


In [ ]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [64]:
acc_score = rf_model.score(X_test_scaled, y_test)
acc_score

0.46375747115252375

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
#deep learning

In [53]:

#https://www.simplilearn.com/tutorials/machine-learning-tutorial/stock-price-prediction-using-machine-learning
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = (len(dummies_df.columns)-1)
hidden_nodes_layer1 =  50
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.LSTM(units=80, return_sequences=True, input_shape=(X_train.shape[1],1)))

# Second hidden layer
nn.add(tf.keras.layers.LSTM(units=30, return_sequences=True))

# Output layer
nn.add(tf.keras.layers.Dense(units=1))


# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 37, 80)            26240     
                                                                 
 lstm_1 (LSTM)               (None, 37, 30)            13320     
                                                                 
 dense (Dense)               (None, 37, 1)             31        
                                                                 
Total params: 39,591
Trainable params: 39,591
Non-trainable params: 0
_________________________________________________________________


In [56]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1469/1469 [==============================] - 102s 63ms/step - loss: -3294135.2500 - accuracy: 0.0000e+00
Epoch 2/100
1469/1469 [==============================] - 93s 63ms/step - loss: -3294138.2500 - accuracy: 0.0000e+00
Epoch 3/100
1469/1469 [==============================] - 92s 63ms/step - loss: -3294136.0000 - accuracy: 0.0000e+00
Epoch 4/100
1469/1469 [==============================] - 93s 63ms/step - loss: -3294141.7500 - accuracy: 0.0000e+00
Epoch 5/100
1469/1469 [==============================] - 94s 64ms/step - loss: -3294133.2500 - accuracy: 0.0000e+00
Epoch 6/100
1469/1469 [==============================] - 94s 64ms/step - loss: -3294137.7500 - accuracy: 0.0000e+00
Epoch 7/100
 240/1469 [===>..........................] - ETA: 1:16 - loss: -3308527.0000 - accuracy: 0.0000e+00

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")